# Advanced Multi GPU Programming with MPI and OpenACC

In this self-paced, hands-on lab, you will learn how to improve a multi GPU MPI+OpenACC program. It is a followup lab of the **Introduction to Multi GPU Programming with MPI and OpenACC** lab. Knowledge on how to program multiple GPUs with MPI and OpenACC is a prerequisite.  The topics covered by this lab are:

* Overlapping communication with computation to hide communication times
* Handling noncontiguous halo updates with a 2D tiled domain decomposition

This Lab was created by Jiri Kraus.

---
Before we begin, let's verify [WebSockets](http://en.wikipedia.org/wiki/WebSocket) are working on your system.  To do this, execute the cell block below by giving it focus (clicking on it with your mouse), and hitting Ctrl-Enter, or pressing the play button in the toolbar above.  If all goes well, you should see some output returned below the grey cell.  If not, please consult the [Self-paced Lab Troubleshooting FAQ](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting) to debug the issue.

In [1]:
print("The answer should be three: " + str(1+2))

The answer should be three: 3


Let's execute the cell below to display information about the GPUs running on the server.

In [2]:
!nvidia-smi

Fri Nov 18 16:07:19 2016       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 980 Ti  Off  | 0000:01:00.0      On |                  N/A |
|  5%   59C    P2    77W / 275W |   1494MiB /  6077MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Motivation

You have propably scaled you program to multiple GPUs to get a faster time to solution or to be able to solve larger problems, but the multi GPU scalability is below expectations or you want to futher improve the scalability. There are two major reasons why the scalability of a multi GPU program does not meet expectations

* Load Imbalances, i.e. some GPUs have to compute signficantly more work than others. This often results in idle GPUs waiting for the busy GPUs to finish.
* Parallel Overhead caused by the necessary coordination of work of multiple GPUs, e.g. halo exchanges.

This lab will cover how to lower the impact of the parallel overhead.

## Scalability Metrics For Success

Like in the **Introduction to Multi GPU Programming with MPI and OpenACC** lab the success of improvments for the multi GPU parallelization is measured with the following metrics. The provided tasks automatically print these metrics out at the end of each lab section/execution.

* Serial Time: $\Large{T_{s}}$  - how long it takes to run the problem with a single GPU
* Parallel Time: $\Large{T_{p}}$  - how long it takes to run the problem on multiple GPUs
* Number of Processors:  $P$  - the number of processors (GPUs) operating in parallel
* Speedup: $\Large{S = \frac{T_{s}}{T_{p}}}$ - How much faster the parallel version is versus the serial version.
 * The ideal speed up is $P$.
* Efficiency: $\Large{E = \frac{S}{P}}$ - How efficiently the processors are being used.
 * The ideal efficiency is $1$.

##Basics

### CUDA-aware MPI

A CUDA-aware MPI implementation allows you to exchange data directly to and from the GPUs buffers involved, avoiding host buffer staging in the user code. For this lab it is sufficient to know that you can directly pass GPU pointers to the MPI routines of a CUDA-aware MPI implementation. If you want to learn more about CUDA-aware MPI I recommend you to read my post on the Parallel Forall blog: 
[An Introduction to CUDA-Aware MPI](http://devblogs.nvidia.com/parallelforall/introduction-cuda-aware-mpi/)

### <code>#pragma acc host_data use_device( A )</code>
In an OpenACC data region a CPU and a GPU copy of each scalar or array exists. Since MPI calls are executed by the CPU the default behavior of an OpenACC program is to use the CPU copy for these calls. In an OpenACC accelerated program however usually the GPU copy of the data is the one operated on so passing the CPU copy into MPI would result in communication of stale data and thus wrong results. The default behavior can be changed with the directive <code>host_data use_device</code>. <code>host_data use_device(A)</code> tells the OpenACC compiler to use the device representation of <code>A</code> in the following code block. Because we are using a CUDA-aware MPI the MPI implementation can handle these and do the halo updates directly to and from GPU memory.

## Structure of this Lab

This lab is broken up into five tasks; instructions for each will be provided in-line below. The "solution" to each task is the starting point of the next task, so you can skip around if you'd like. In addition, reference solutions are provided for each task. You can find them by looking for files matching \*.solution\*. You can build and run the solution with the make target <code>task?.solution</code>, e.g. for task 1: <code>make -C C task1.solution</code>.

Instructions for downloading this IPython Notebook, as well as a .zip file of the source you worked on, are provided at the bottom of the lab in the <a href="#post-lab">Post Lab</a> section.


## 2D Poisson Solver

The code used in this lab is a Jacobi solver for the 2D Poisson equation on a rectangle:

<div align="center">

$\LARGE{\Delta A(x,y) = \LARGE{e^{-10 * (x^2 + y^2)}} \; \forall (x,y) \in \Omega\backslash\delta\Omega}$

</div>

With periodic boundary conditions.

Given a 2D grid of vertexes, the solver attempts to set every vertex equal to the average of neighboring vertices.  It will iterate until the system converges to a stable value. So in each iteration of the Jacobi solver for all interior vertices 

<div align="center"><img src="files/laplace2d.PNG" width="30%" />$\LARGE{A_{k+1}(i,j)} = \frac{rhs(i,j) - A_{k}(i-1,j) + A_{k}(i + 1,j) + A_{k}(i,j-1) + A_{k}(i,j+1)}{4}$</div>

is applied and then the periodic boundary conditions are handled by copying

1. the values of the first interior row to the bottom boundary and the last interior row to the top boundary:
    <div align="center">
    <img src="files/jacobi_iteration_B.PNG" width="30%" />
    </div>
2. the values of the first interior column to the right boundary and the last interior column to the left boundary:
    <div align="center">
    <img src="files/jacobi_iteration_C.PNG" width="30%" />
    </div>

## Using the NVIDIA Visual Profiler (NVVP)

As described in [CUDA Pro Tip: Profiling MPI Applications](http://devblogs.nvidia.com/parallelforall/cuda-pro-tip-profiling-mpi-applications/) <code>nvprof</code> can be used to generate profiles of MPI+OpenACC applications. Execute the <code>make -C C task?.profile</code> in the one of the cells below to generate a profile for each MPI rank executing your solution of the selected task.

To view the generated profiles we'll be using the NVIDIA Visual Profiler (NVVP) tool which comes standard with the CUDA Toolkit software.  To launch the tool please <a href="/vnc" onclick="window.open(this.href, 'ProfilerVNC',
'left=20,top=20,width=1290,height=730,toolbar=1,resizable=0'); return false;">click here</a> which will open a new browser window.  **Note that it may take a few seconds for NVVP to start.**

After NVVP has started, import the generated profiles by clicking on "File" and then "Import...". In the dialog select "nvprof" and "Multiple Processes". Browse to <code>ubuntu/notebook/C/task?</code> and select <code>poisson2d.[0-3].nvvp</code>.

If you've never used NVVP before or if you want to read more about you can [click here](https://developer.nvidia.com/nvidia-visual-profiler) for more information.

# Lab Tasks

** This is a long lab, so please pay attention to your time.**  You have 120 minutes of access time from when the lab connection information was presented to you.  You may want to pre-download the IPython Notebook and source in the <a href="#post-lab">Post Lab</a> before continuing.

## Task #1

If you execute the initial version with `make -C C task1` in the cell below you will see that the domain decomposition used to distribute the work across multiple GPUs delivered a speed-up but does not attain optimal efficiency. This is because of the time that is needed (spent) to carry out the halo updates using MPI. This wasted time is called "parallel overhead" because it is a step not necessary for execution with a single GPU. We can lower the parallel overhead by doing computations in parallel with the MPI communication and therefore hide the communication time. In the case of our Jacobi solver, this is best done by splitting each domain into a boundary part (which updates all values that we need to communicate) and an inner part. By doing this split, we can start the MPI communication after the boundary part has finished, and let it run in parallel with the inner part:

<div align="center">

<img src="files/overlapping_mpi_compute.png" width="80%" />

</div>

In OpenACC this can be done by using the async clause on a kernels region as outlined below.

```c
#pragma acc kernels
for ( ... )
     //Process boundary
#pragma acc kernels async
for ( ... )
     //Process inner domain 

#pragma acc host_data use_device ( A )
{
  //Exchange halo with top and bottom neighbor
  MPI_Sendrecv( A…);
  //…
}
//wait for iteration to finish
#pragma acc wait
```

In this task you should apply this approach to the copy loop of the Jacobi solver. Look out for <code>TODO</code> in <code>C/task1/poisson2d.c</code>. These TODOs will guide you through the following steps:

* Split the copy loop into its constituent halo and bulk parts.
* Start the computation of the bulk part asynchronously.
* Wait for the bulk part to complete at the end of the iteration.

To compile and run simply issue <code>make -C C task1</code> as given in the cell below.

The following reference might be interesting for you:

* API documentation for MPI from the OpenMPI website [https://www.open-mpi.org/doc/v1.8](https://www.open-mpi.org/doc/v1.8).
* [OpenACC 2.0 Quick Reference Guide](http://104.239.134.127/sites/default/files/213462%2010_OpenACC_API_QRG_HiRes.pdf)

In [ ]:
!make -C C task1

At the end of the output you will see a output similar to this:
```
Num GPUs: 4.
4096x4096: 1 GPU:   5.8220 s, 4 GPUs:   1.6919 s, speedup:     3.44, efficiency:    86.03%
```
So compared to the initial results
```
Num GPUs: 4.
4096x4096: 1 GPU:   5.8211 s, 4 GPUs:   1.7709 s, speedup:     3.29, efficiency:    82.18%
```
The parallel efficiency increased by about 4%.

## Domain Decomposition Stragegies

There are three major options for breaking up our 2D grid of vertexes, or domain, to parallelize the work across multiple GPUs. The halo region shown in light green in the images is the data that needs to be shared among the GPUs working on the problem.

<div style="display: inline;"><br><img src="files/decomp_tiles.PNG" align="left" width="15%" />
<ul style="position: relative; left: 15px; vertical-align: middle;">Minimizes surface area/volume ratio:
<li style="position: relative; left: 30px;">Communicate less data</li>
<li style="position: relative; left: 30px;">Optimal for bandwidth bound communication</li></ul></div>

<div style="clear: left;"><br><img src="files/decomp_vertical_stripes.PNG" align="left" width="15%" />
<ul style="position: relative; left: 15px;">Minimizes number of neighbors:
<li style="position: relative; left: 30px;">Communicate to fewer neighbors</li>
<li style="position: relative; left: 30px;">Optimal for latency bound communication</li>
Contiguous if data is [column-major](https://en.wikipedia.org/wiki/Column-major_order)</ul></div>

<div style="clear: left;"><br><img src="files/decomp_horizontal_stripes.PNG" align="left" width="15%" />
<br><ul style="position: relative; left: 15px;">Minimizes number of neighbors:
<li style="position: relative; left: 30px;">Communicate to fewer neighbors</li>
<li style="position: relative; left: 30px;">Optimal for latency bound communication</li>
Contiguous if data is [row-major](https://en.wikipedia.org/wiki/Row-major_order)</ul></div>

<div style="clear: left;"><br><br></div>

The initial version of the Jacobi solver used in this lab applied a domain decomposition using Horizontal Stripes as row-major order is used in C/C++. Since on the cloud nodes available for this lab only 4 GPUs are available this is also the optimal choice because:

* it minimizes the number of neighbors we need to comunicate with
* with only 4 GPUs the amount of data that needs to be communicated with all neighbors is the same for a domain decomposition with stripes and tiles

However depending on the amount of data a given applicaiton needs to exchange and hardware properties like network bandwidth a tiled domain decompostion can be better when more GPUs are used. So even if it does not give a benefit for the available number of GPUs in this lab and for the given solver the remaining tasks modify the given solver to apply a 2D domain decomposition step by step.

## Noncontiguous halo updates

A domain decomposition with tiles requires to exchange noncontigous data between GPUs. Exchanging noncontiguous data between GPUs is inefficient because it requires to send many small messages. To avoid that a common strategy is to use staging buffers to send all data that needs to be exchanged with a neighbor in a single larger message. In case of our 2D Jacobi solver we need four staging buffers:

* <code>to_left</code>: to gather data which needs to be send to left neighbor
* <code>to_right</code>: to gather data which needs to be send to right neighbor
* <code>from_left</code>: to receive data from left neighbor
* <code>from_right</code>: to receive data from right neighbor

Then before starting MPI communication with the left and right neighbor data is gathered into <code>to_left</code> and <code>to_right</code> and after the MPI communication the received data is scattered into the working buffer from <code>from_left</code> and <code>from_right</code>:

<div align="center">

<img src="files/jacobi_halo_update_left_right.png" width="80%" />

</div>

Using MPI datatypes is an alternative to application managed staging buffers. Depending on the used MPI implementation that can even be more efficient because the MPI implementation can do the packing and unpacking in its internal pipeline and therefore better hide the packing and unpacking time.

If we treat the left most rank responsible for a certain set of rows as the right neighbour of the right most rank responsible for the same set of rows and the right most rank as the left neighbour of the left most rank doing the left/right halo update will also handle the periodic boundary conditions and therefore the loop handling those can be removed.

## Task #2

The purpose of this and the following tasks is to change the domain decomposition from horizontal stripes to a tiled domain decomposition. To let you focus on the logical domain decomposition and GPU to GPU communication the data here is still fully replicated on each GPU. (This is something one would normally not do, as it has a significant storage overhead, but it avoids some boilerplate code that should not be part of this lab.). The process of this transformation is brocken up into multiple smaller steps. In Task 2 2D MPI ranks should be introduced.

Like in Task #1 you should look out for <code>TODO</code> in <code>C/task2/poisson2d.c</code>. These will guide you through the following steps:

* Use the provided method <code>size_to_2Dsize</code> to derive 2D MPI ranks and size from the given 1D MPI rank and size.
* Map the 2D rank back to the MPI rank to be used in <code>MPI_Sendrecv</code>

The MPI standard allows to create communicators with cartesian topology information attached. These communicators can be used to accheive the same (<code>MPI_Cart_create</code>). For educational purposes this is done manually in this lab.

[Click here to see hints](#Task-#2---Hints)

In [ ]:
!make -C C task2

At the end of the output you will see a output similar to this:
```
Num GPUs: 4 with a (2,2) layout.
4096x4096: 1 GPU:   5.8207 s, 4 GPUs:   3.1414 s, speedup:     1.85, efficiency:    46.32%
```
Since we are replicating the work of the two GPUs responsible for the same set of rows its expected that we get about half the parallel efficiency compared to the origingal code.

## Task #3

In Task 3 the staging buffers <code>to_left,from_left,to_right,from_right</code> should be used to handle the periodic boundary conditions.

Like in the other tasks you should look out for <code>TODO</code> in <code>C/task3/poisson2d.c</code>. These will guide you through the following steps:

* Gather data to be copied from left most inner column to right boundary in <code>to_left</code>
* Gather data to be copied from right most inner column to left boundary in <code>to_right</code>
* Scatter data to be copied from left most inner column to right boundary from <code>from_right</code>
* Scatter data to be copied from right most inner column to left boundary from <code>from_left</code>


In [ ]:
!make -C C task3

At the end of the output you will see a output similar to this:
```
Num GPUs: 4 with a (2,2) layout.
4096x4096: 1 GPU:   5.8309 s, 4 GPUs:   3.1608 s, speedup:     1.84, efficiency:    46.12%
```
Since we are still replicating the work of the two GPUs responsible for the same set of rows its expected that we do not see an improvment in parallel efficiency compared to the last task. This will be fixed in the next task.

## Task #4

In Task 4 the actual domain decomposition of the rows should be applied.

Like in the other tasks you should look out for <code>TODO</code> in <code>C/task4/poisson2d.c</code>. These will guide you through the following steps:

* set first (<code>ix_start</code>) and last (<code>ix_end<code>) column to be processed by each rank
* replace the copy <code>from_left</code>,<code>to_right</code>,<code>from_right</code>,<code>to_left</code> loop with MPI communication.


In [ ]:
!make -C C task4

At the end of the output you will see a output similar to this:
```
Num GPUs: 4 with a (2,2) layout.
4096x4096: 1 GPU:   5.8209 s, 4 GPUs:   1.8447 s, speedup:     3.16, efficiency:    78.88%
```
As said above its expected that the parallel efficiency does not increase compared to the initial version using a domain decomposition with stripes. With only 4 GPUs the amount of data each GPU needs to exchange with its neighbors is the same for a tiled domain decomposition and a domain decomposition with stripes. Furthermore the tiled domain decomposition has a higher parallel overhead because of the necessary gather and scatter steps. With more GPUs this can change and doing the gather and scatter steps will pay off and lead to better scalabilty of the code.

## Optional: Task #5

This is an optional task to apply communication computation overlap to the tiled domain decomposition version as it was done above in task 1 for the inital version using a domain decompostion with stripes. The aim is hide as much of the parallel overhead (MPI, gather, scatter) as possible. As in the other tasks follow the TODOs in <code>C/task5/poisson2d.c</code>.

In [ ]:
!make -C C task5

## Summary

In this lab you have learned how to improve the scalability of a MPI+OpenACC multi GPU application by hiding communicaiton times and applying a 2D tiled domain decomposition to decrease the amount of data that needs to be exchanged between GPUs.

<a id="post-lab"></a>
## Post-Lab

Finally, don't forget to save your work from this lab before time runs out and the instance shuts down!!

1. Save this IPython Notebook by going to `File -> Download as -> IPython (.ipynb)` at the top of this window
2. You can execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
rm -f advanced_multi_gpu_mpi_openacc_files.zip
zip -r advanced_multi_gpu_mpi_openacc_files.zip C FORTRAN

**After** executing the above cell, you should be able to download the zip file [here](files/advanced_multi_gpu_mpi_openacc_files.zip)

<a id="References/Further Reading"></a>
## References/Further Reading

* Learn more at the [CUDA Developer Zone](https://developer.nvidia.com/category/zone/cuda-zone).
* If you have an NVIDIA GPU in your system, you can download and install the [CUDA tookit](https://developer.nvidia.com/cuda-toolkit).
* Take the fantastic online and **free** Udacity [Intro to Parallel Programming](https://www.udacity.com/course/cs344) course which uses CUDA C.
* Search or ask questions on [Stackoverflow](http://stackoverflow.com/questions/tagged/cuda) using the cuda tag
* Read the GPU Computing developer blog [Parallel Forall](http://devblogs.nvidia.com/parallelforall/)

<a id="FAQ"></a>
---
# Lab FAQ

Q: I'm encountering issues executing the cells, or other technical problems?<br>
A: Please see [this](https://developer.nvidia.com/self-paced-labs-faq#Troubleshooting) infrastructure FAQ.

## Hints

### Task #2 - Hints
#### Hint #1
Modulo (<code>%</code>) and devide (<code>/</code>) are your friend.

#### Hint #2
You can use the following to compute 2D MPI rank and size from the provided 1D MPI rank and size.
```
dim2 size2d = size_to_2Dsize(size);
int sizex = size2d.x;
int sizey = size2d.y;
assert(sizex*sizey == size);
    
int rankx = rank%sizex;
int ranky = rank/sizex;
```

[Return to Task #2](#Task-#2)